In [1]:
# base modules
import os
import sys
import copy

# for manipulating data
import numpy as np
import pandas as pd
import math
import dill
import gc

# for Machine Learning
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, plot_tree

# for visualization
from IPython.display import display
from matplotlib import pyplot as plt

In [2]:
# path to repo
path_to_repo = os.path.dirname(os.getcwd())
sys.path.insert(0, os.path.join(path_to_repo, 'src'))

path_to_repo

'/Users/jiaokan/Workshop/Machine Learning in Practice'

In [3]:
# custom module
from mlpcourse.utils import *

In [4]:
# number of CPUs on my machine:
os.cpu_count()

8

### Load and preprocess data

We'll work with the huge data set of the Corporation Favorita Grocery Sales Forecasting kaggle competition from 2018.

Data avaiable [here](https://www.kaggle.com/c/favorita-grocery-sales-forecasting/data)

In [5]:
# types = {'id': 'int64',
#          'int_nbr': 'int32',
#          'store_nbr': 'int8',
#          'unit_sales': 'float32',
#          'onpromotion': 'object'}  # boolean with missing values

In [6]:
# df_all = pd.read_csv('../data/train.csv',  parse_dates=['date'], dtype=types)
# path_to_dataset = os.path.join(
#     path_to_repo, "data", 'raw_groceries_full')

# df_all = pd.read_feather(path_to_dataset)

# df_chunks = pd.read_csv('../data/train.csv',
#                         parse_dates=['date'], dtype=types, chunksize=100000000)
# df_all = pd.concat(df_chunks)

In [7]:
# df_all.onpromotion.fillna(False, inplace=True)
# df_all.onpromotion = df_all.onpromotion.map({'False': False, 'True': True})
# df_all.onpromotion = df_all.onpromotion.astype(bool)

# %time df_all.to_feather('raw_groceries')

In [8]:
# %time df_all.describe(include='all')

In [9]:
# df_all.unit_sales = np.log1p(np.clip(df_all.unit_sales, 0, None))
# cutoff neg values of unit_sales, take log + 1

In [10]:
# %%time
# add_datepart(df_all, 'date')

In [11]:
# df_all

In [12]:
# df_all.to_feather('raw_groceries')

In [13]:
# items = pd.read_csv('items.csv')
# items.shape

In [14]:
# items.head()

In [15]:
# items.dtypes

In [16]:
# items.item_nbr.unique().shape

In [17]:
# items.family.unique()

In [18]:
# train_cats(items)

In [19]:
# items.family.cat.codes.unique()

In [20]:
# items.family = items.family.cat.codes

In [21]:
# items.to_feather('items')

In [22]:
# itms = df_all['item_nbr'].to_numpy()
# itms.shape

In [23]:
# itms[:10]

In [24]:
# fam = dict(zip(items.item_nbr, items.family))
# cls = dict(zip(items.item_nbr, items['class']))
# per = dict(zip(items.item_nbr, items.perishable))

In [25]:
# def apply_dict_to_array(conversion_dict, input_array):
#     k = np.array(list(conversion_dict.keys()))
#     v = np.array(list(conversion_dict.values()))

#     mapping_ar = np.zeros(k.max()+1, dtype=v.dtype)
#     mapping_ar[k] = v
#     return mapping_ar[input_array]

In [26]:
# fam_col = apply_dict_to_array(fam, itms)
# cls_col = apply_dict_to_array(cls, itms)
# per_col = apply_dict_to_array(per, itms)

In [27]:
# df_all['family'] = fam_col
# df_all['class'] = cls_col
# df_all['perishable'] = per_col

In [28]:
# df_all.to_feather('raw_groceries')

In [29]:
# df_all = pd.read_feather('raw_groceries')

In [30]:
# df_all.head()

In [31]:
# stores = pd.read_csv('stores.csv')

In [32]:
# stores.shape

In [33]:
# stores.dtypes

In [34]:
# stores.head()

In [35]:
# train_cats(stores)

In [36]:
# stores.city = stores.city.cat.codes
# stores.state = stores.state.cat.codes
# stores.type = stores.type.cat.codes

In [37]:
# city = dict(zip(stores.store_nbr, stores.city))
# state = dict(zip(stores.store_nbr, stores.state))
# type = dict(zip(stores.store_nbr, stores.type))

In [38]:
# stores_in_db = df_all['store_nbr'].to_numpy()
# stores_in_db.shape

In [39]:
# city_col = apply_dict_to_array(city, stores_in_db)
# state_col = apply_dict_to_array(state, stores_in_db)
# type_col = apply_dict_to_array(tpe, stores_in_db)

In [40]:
# df_all['city'] = city_col
# df_all['state'] = state_col
# df_all['type'] = tpe_col

In [41]:
# df_all.to_feather('raw_groceries')

In [42]:
# df_all = pd.read_feather('raw_groceries')
# df_all.head()

features considered by kaggle winners:
- store x item
- store x class
- ...

plus a fine feature engineering (including computing means of products prices over various time windows)

In [43]:
# df_all['store_nbr x item_nr'] = df_all['store_nbr'] * df_all['item_nbr']
# df_all['store_nbr x class'] = df_all['store_nbr'] * df_all['class']

In [44]:
# df_all.head()

In [45]:
# df_all.to_feather('raw_groceries_full')

### Fit

In [46]:
# df_all = pd.read_feather('raw_groceries_full')

In [47]:
# df_all.shape

Kaggle first rankers used only 2017 data...

In [48]:
# df = df_all[df_all['Year'] == 2017].copy()

In [49]:
path_to_dataset = os.path.join(
    path_to_repo, "data", 'raw_groceries_2017')

df = pd.read_feather(path_to_dataset)

In [50]:
df.shape

(23808261, 27)

In [52]:
def split_vals(a, n): return a[:n].copy(), a[n:].copy()

In [53]:
n_valid = 3370464
n_trn = len(df) - n_valid
train, valid = split_vals(df, n_trn)
train.shape, valid.shape

((20437797, 27), (3370464, 27))

In [54]:
def nwrmsle(y_true, y_pred, w):
    """x_per bool array with value 1 if item is perishable, 0 else"""
    return math.sqrt((w*((y_true-y_pred)**2)).sum() / w.sum())
# def rmse(y_true, y_pred):
#     return math.sqrt(((y_true - y_pred)**2).mean())

In [55]:
def print_score(m, X_train, y_train, X_valid, y_valid):
    w_trn = 1 + .25*X_train[:, -1]  # extract `perishable` column
    w_val = 1 + .25*X_valid[:, -1]  # extract `perishable` column
    print('NWRMSLE on train set: {:.4f}'.format(
        nwrmsle(m.predict(X_train), y_train, w_trn)))
    print('NWRMSLE on valid set: {:.4f}'.format(
        nwrmsle(m.predict(X_valid), y_valid, w_val)))
    print('R^2 on train set: {:.4f}'.format(m.score(X_train, y_train)))
    print('R^2 on valid set: {:.4f}'.format(m.score(X_valid, y_valid)))
    if hasattr(m, 'oob_score_'):
        print('R^2 on oob set: {:.4f}'.format(m.oob_score_))
    return

#### Train on 2017 data

NB: do NOT use OOB_score (would be slow to evaluate performance of a tree outside the small subset it was trained on)

In [56]:
train.shape

(20437797, 27)

In [57]:
%%time
trn, y, _ = proc_df(train, 'unit_sales')

CPU times: user 1.15 s, sys: 4.85 s, total: 6 s
Wall time: 9.45 s


In [58]:
%%time
val, y_val, _ = proc_df(valid, 'unit_sales')

CPU times: user 191 ms, sys: 302 ms, total: 493 ms
Wall time: 528 ms


In [59]:
model = RandomForestRegressor(n_jobs=-1,
                              n_estimators=10,
                              max_samples=10_000)

In [60]:
%prun model.fit(trn,y)

         33150 function calls (32805 primitive calls) in 41.569 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     3138   37.850    0.012   37.850    0.012 {built-in method time.sleep}
        1    1.301    1.301    1.304    1.304 managers.py:1707(_interleave)
       14    1.214    0.087    1.214    0.087 {method 'astype' of 'numpy.ndarray' objects}
        5    0.644    0.129    0.644    0.129 {method 'reduce' of 'numpy.ufunc' objects}
        9    0.271    0.030   38.172    4.241 parallel.py:1746(_retrieve)
        1    0.062    0.062    0.062    0.062 {built-in method numpy.ascontiguousarray}
        2    0.059    0.029    2.640    1.320 validation.py:718(check_array)
        1    0.034    0.034    0.034    0.034 {built-in method _imp.create_dynamic}
     3147    0.029    0.000    0.029    0.000 parallel.py:1719(_wait_retrieval)
        1    0.024    0.024   41.568   41.568 base.py:1456(wrapper)
3406/3357    0.013    0

We see that sklearn spends a lot of time converting our input df (trn) into a numpy.ndarray, so let's convert it before calling the fit method:

In [61]:
%time x = np.array(trn, dtype=np.float32)

CPU times: user 18.5 s, sys: 13.5 s, total: 32 s
Wall time: 1min 56s


In [62]:
%time x_val = np.array(val, dtype=np.float32)

CPU times: user 2.87 s, sys: 421 ms, total: 3.29 s
Wall time: 3.51 s


Let's run lots of trees on small subsets (with `max_samples` set small):

In [63]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=200,
                          # max_depth=10,
                          min_samples_leaf=100,
                          max_features=0.75,
                          max_samples=20_000)
%time m.fit(x,y)

CPU times: user 1min 7s, sys: 2min 7s, total: 3min 14s
Wall time: 58 s


RandomForestRegressor(max_features=0.75, max_samples=20000,
                      min_samples_leaf=100, n_estimators=200, n_jobs=-1)

In [64]:
print_score(m, x, y, x_val, y_val)

NWRMSLE on train set: 0.7635
NWRMSLE on valid set: 0.7657
R^2 on train set: 0.1950
R^2 on valid set: 0.1850


In [65]:
gc.collect()

115

In [66]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=500,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=50_000)
%time m.fit(x,y)

CPU times: user 8min 45s, sys: 46min 7s, total: 54min 52s
Wall time: 18min 28s


RandomForestRegressor(max_features=0.75, max_samples=50000, min_samples_leaf=50,
                      n_estimators=500, n_jobs=-1)

In [67]:
print_score(m, x, y, x_val, y_val)

NWRMSLE on train set: 0.7360
NWRMSLE on valid set: 0.7409
R^2 on train set: 0.2550
R^2 on valid set: 0.2404


In [68]:
gc.collect()

115

In [69]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=50,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=1_000_000)
%time m.fit(x,y)

CPU times: user 16min 1s, sys: 39min 48s, total: 55min 50s
Wall time: 16min 39s


RandomForestRegressor(max_features=0.75, max_samples=1000000,
                      min_samples_leaf=50, n_estimators=50, n_jobs=-1)

In [70]:
print_score(m, x, y, x_val, y_val)
gc.collect()

NWRMSLE on train set: 0.6343
NWRMSLE on valid set: 0.6518
R^2 on train set: 0.4497
R^2 on valid set: 0.4196


115

In [71]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=10,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=5_000_000)
%time m.fit(x,y)

CPU times: user 13min 45s, sys: 22min 31s, total: 36min 16s
Wall time: 11min 20s


RandomForestRegressor(max_features=0.75, max_samples=5000000,
                      min_samples_leaf=50, n_estimators=10, n_jobs=-1)

In [72]:
print_score(m, x, y, x_val, y_val)
gc.collect()

NWRMSLE on train set: 0.5560
NWRMSLE on valid set: 0.5883
R^2 on train set: 0.5748
R^2 on valid set: 0.5267


115

In [73]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=5,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=10_000_000)
%time m.fit(x,y)

CPU times: user 10min 21s, sys: 3min 33s, total: 13min 54s
Wall time: 3min 38s


RandomForestRegressor(max_features=0.75, max_samples=10000000,
                      min_samples_leaf=50, n_estimators=5, n_jobs=-1)

In [74]:
print_score(m, x, y, x_val, y_val)
gc.collect()

NWRMSLE on train set: 0.5285
NWRMSLE on valid set: 0.5674
R^2 on train set: 0.6170
R^2 on valid set: 0.5606


115

In [75]:
x.shape

(20437797, 26)

In [76]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=5,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=0.7)
%time m.fit(x,y)

CPU times: user 13min 40s, sys: 9min 22s, total: 23min 2s
Wall time: 6min 57s


RandomForestRegressor(max_features=0.75, max_samples=0.7, min_samples_leaf=50,
                      n_estimators=5, n_jobs=-1)

In [77]:
print_score(m, x, y, x_val, y_val)
gc.collect()

NWRMSLE on train set: 0.5145
NWRMSLE on valid set: 0.5569
R^2 on train set: 0.6368
R^2 on valid set: 0.5761


115

and we'd be in top 1000...

In [78]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=5,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=0.8)
%time m.fit(x,y)

CPU times: user 16min 13s, sys: 8min 19s, total: 24min 33s
Wall time: 7min 4s


RandomForestRegressor(max_features=0.75, max_samples=0.8, min_samples_leaf=50,
                      n_estimators=5, n_jobs=-1)

In [79]:
print_score(m, x, y, x_val, y_val)
gc.collect()

NWRMSLE on train set: 0.5090
NWRMSLE on valid set: 0.5540
R^2 on train set: 0.6450
R^2 on valid set: 0.5814


115

In [80]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=10,
                          # max_depth=10,
                          min_samples_leaf=50,
                          max_features=0.75,
                          max_samples=0.6)
%time m.fit(x,y)

KeyboardInterrupt: 

In [ ]:
print_score(m, x, y, x_val, y_val)
gc.collect()

#### Train and validate on kaggle winner's data

See winner's solution [here](https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting/discussion/47582)

train: 20170614 - 20170719 (id=118867503 to id=122667528)

valid: 20170726 - 20170810 (id=123296175 to id=124975583)

In [6]:
df = pd.read_feather('raw_groceries_full')

In [7]:
df_train = df[(df['id'] >= 118867503) & (df['id'] < 122667528)]
df_test = df[(df['id'] >= 123296175) & (df['id'] < 124975583)]
del df

In [ ]:
df_train.head()

In [ ]:
df_train.shape, df_test.shape

In [9]:
trn, y, _ = proc_df(df_train, 'unit_sales')
val, y_val, _ = proc_df(df_test, 'unit_sales')

In [ ]:
%time x = np.array(trn, dtype=np.float32)
# try also np.float16

In [ ]:
%time x_val = np.array(val, dtype=np.float32)

In [ ]:
gc.collect()

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=25,
                          # max_depth=10,
                          min_samples_leaf=10,
                          max_features=0.9,
                          max_samples=0.5,
                         )
%time m.fit(x,y)

In [ ]:
%time print_score(m,x,y,x_val,y_val)
gc.collect()

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=25,
                          # max_depth=10,
                          min_samples_leaf=10,
                          max_features=0.9,
                          # bootstrap = True
                          max_samples=0.9,
                         )
%time m.fit(x,y)

In [ ]:
%time print_score(m,x,y,x_val,y_val)
gc.collect()

In [ ]:
preds = np.stack([tree.predict(x_val)
                 for tree in m.estimators_])  # or np.asarray()
w_val = 1 + .25*x_val[:, -6]  # extract `perishable` column

plt.plot([
    # r2_score(y_val, np.mean(preds[:num_trees], axis = 0))
    nwrmsle(np.mean(preds[:num_trees], axis=0), y_val, w_val)
    for num_trees in range(1, 26)
])

In [ ]:
m = RandomForestRegressor(n_jobs=-1,
                          n_estimators=30,
                          # max_depth=10,
                          min_samples_leaf=5,
                          max_features=0.9,
                          # bootstrap = True
                          max_samples=0.9,
                         )
%time m.fit(x,y)

In [ ]:
%time print_score(m,x,y,x_val,y_val)
gc.collect()